# Initial setup

Let's import the required libraries and set up global variables for the rest of the script.

In [15]:
# coding: utf-8
!pip install tqdm
import csv
import os
import re
import shutil
import string
import zipfile
import sys
from collections import defaultdict
from lxml import objectify
import codecs
import nltk
import pandas as pd
import requests
import tarfile
import subprocess
import platform
import time
from tqdm import tqdm as progressbar # pandas df usage: 'for row in progressbar(df.itertuples(), total=df.shape[0])'

import glob

Helper function to create a directory under the specified path, gracefully handling errors.

In [2]:
def __mkdir(*args):
    path = os.path.join(*args)
    try: 
        os.makedirs(path)
    except OSError:
        if not os.path.isdir(path):
            raise
    return path

In [16]:
# Create the project directory holding the downloaded data, serialized dataframes and MetaMap install.
# working_dir = __mkdir(os.path.expanduser("~"), "Medframes")

# Set working directory as the current directory of the ipython notebook
working_dir = os.getcwd()
data_dir = __mkdir(working_dir, "data")
print("Working directory: %s" % working_dir)

Working directory: /Users/Lo/Work/CS109Project


# Load MetaMap & MedEx processing results

In [15]:
mm = pd.read_pickle(os.path.join(data_dir, 'mm.pckl'))
mm.head()

,nct_id,criteria_id,ngram_index,score,term,cui,stype,cid
0,NCT00001149,0,0,11.49,Historical aspects qualifier,C0019665,inpr,[x.x.x]
1,NCT00001149,0,0,8.34,History,C0019664,ocdi,[K01.400]
2,NCT00001149,0,1,17.80,Seizures,C0036572,sosy,"[C10.228.140.490.631, C10.597.742, C23.888.592..."
3,NCT00001149,0,3,8.34,Time,C0040223,tmco,[G01.910]
4,NCT00001149,0,7,17.80,Seizures,C0036572,sosy,"[C10.228.140.490.631, C10.597.742, C23.888.592..."


In [5]:
medex = pd.read_pickle(os.path.join(data_dir, 'ct_medex.pckl'))
medex.head()

,Lemmas,NctId,Ngrams,Tags,Tokens,criteria_id,MedEx
0,"[(History, history), (of, of), (uncontrolled, ...",NCT00001149,"[((history, NN),), ((seizure, NNS),), ((presen...","[(History, NN), (of, IN), (uncontrolled, VBN),...","[History, of, uncontrolled, seizures, at, the,...",0,
1,"[(Seizure, seizure), (frequency, frequency), (...",NCT00001149,"[((seizure, NN),), ((frequency, NN),), ((histo...","[(Seizure, NN), (frequency, NN), (by, IN), (hi...","[Seizure, frequency, by, history, must, be, su...",1,
2,"[(Patients, patient), (of, of), (any, any), (a...",NCT00001149,"[((patient, NNS),), ((age, NN),), ((patient, N...","[(Patients, NNS), (of, IN), (any, DT), (age, N...","[Patients, of, any, age, may, be, accepted]",2,
3,"[(Patients, patient), (and, and), (parents, pa...",NCT00001149,"[((patient, NNS),), ((parent, NNS),), ((guardi...","[(Patients, NNS), (and, CC), (parents, NNS), (...","[Patients, and, parents, or, guardians, if, ap...",3,
4,"[(Patients, patient)]",NCT00001192,"[((patient, NNS),)]","[(Patients, NNS)]",[Patients],4,


# Merging datasets by criteria_id

In [23]:
merged = mm.copy()
merged["MedEx"] = ''

print "Merging progress: ",
for i in merged.index[1000:2000]:
    if i % 500 == 0:
        print "{:.2f}".format(float(i)/len(merged)),
    merged.MedEx[i] = medex.MedEx[merged.criteria_id[i]]
print
print "Merging completed."

merged.head()

NameError: name 'mm' is not defined

In [ ]:
merged.to_pickle(os.path.join(data_dir, 'ct_merged.pckl'))

In [24]:
webapp_data_dir = os.path.join(working_dir, "search_server/data/")
terms = pd.read_pickle(os.path.join(webapp_data_dir, 'terms.pckl'))
trials = pd.read_pickle(os.path.join(webapp_data_dir, 'trials.pckl'))
medex = pd.read_pickle(os.path.join(data_dir, 'medex.pckl'))


In [25]:
#terms.cui
pd.merge(terms, medex, how='inner', on=['cui'])

,cui,nct_ids,num_docs,term,NctId,criteria_id,drug_name,brand_name,drug_form,strength,dose,route,frequency,duration,neccessity,rxnorm_cui,rxnorm_cui_generic,generic_name
0,C0003299,"[NCT00001932, NCT00004758, NCT00004776, NCT000...",244,Antiepileptic Agents,NCT00001932,108,antiepileptic,,,,,,,,,932,932,antiepileptic
1,C0003299,"[NCT00001932, NCT00004758, NCT00004776, NCT000...",244,Antiepileptic Agents,NCT00004758,192,antiepileptic,,,,,,,1 month,,932,932,antiepileptic
2,C0003299,"[NCT00001932, NCT00004758, NCT00004776, NCT000...",244,Antiepileptic Agents,NCT00004776,200,antiepileptics,,,,,,,,,932,932,antiepileptics
3,C0003299,"[NCT00001932, NCT00004758, NCT00004776, NCT000...",244,Antiepileptic Agents,NCT00004776,200,antiepileptic,,,,,,,,,932,932,antiepileptic
4,C0003299,"[NCT00001932, NCT00004758, NCT00004776, NCT000...",244,Antiepileptic Agents,NCT00028262,395,antiepileptic,,,,,orally,,6 months,,932,932,antiepileptic
5,C0003299,"[NCT00001932, NCT00004758, NCT00004776, NCT000...",244,Antiepileptic Agents,NCT00029536,401,antiepileptic,,,,,,,2 months,,932,932,antiepileptic
6,C0003299,"[NCT00001932, NCT00004758, NCT00004776, NCT000...",244,Antiepileptic Agents,NCT00043875,490,antiepileptic drugs,,,,,,,,,932,932,antiepileptic drugs
7,C0003299,"[NCT00001932, NCT00004758, NCT00004776, NCT000...",244,Antiepileptic Agents,NCT00043875,490,antiepileptic drugs,,,,,,,,,932,932,antiepileptic drugs
8,C0003299,"[NCT00001932, NCT00004758, NCT00004776, NCT000...",244,Antiepileptic Agents,NCT00043901,506,antiepileptic,,,,,,,,,932,932,antiepileptic
9,C0003299,"[NCT00001932, NCT00004758, NCT00004776, NCT000...",244,Antiepileptic Agents,NCT00043914,529,antiepileptic,,,,,,,,,932,932,antiepileptic
